# Organize directories

The first step in training a ML model on sequences of images acquired from the ProtectCode app is to ensure the directory structure is set up for subsequent data loading. 

These commands assume the following: 

1) That the top directory for the training data and this notebook are both located in ~/src/YPB-AI/ on the EC2 instance. To get a zipped directory from S3 to that directory on your EC2 instance, you can run the following bash command: "aws s3 cp s3://your-bucket-name/yourZipFile.zip ~/src/YPB-AI/yourZipFile.zip". And to unzip it, you can run "unzip yourZipFile.zip"

2) That the jupyter notebook server was opened from the ~/src/ directory on the EC2 instance via the command "jupyter notebook --ip=0.0.0.0 --no-browser".

3) That underneath the top directory (which I usually call "Training" or something of the like), there are a series of subdirectories named and organized by classification label. For instance, each subdirectory that contains true negative image sequences, according to my implementation, contains the word "Negative". And subdirectories with true positives do not contain the word "Negative". 



In [1]:
import numpy as np
from glob import glob
import os, sys
import re

Set the name of your top data directory below. 

In [2]:
# topDirName = 'Training'
# topDirName = 'Training2'
topDirName = 'bamData_no_pillBottle'

Lump together all subdirectories with the same classification label. 

In [12]:
#Assumes current working directory is ~/src/YPB-AI
allSamples = np.array(glob(topDirName + '/*'))
os.system('mkdir ' + topDirName + '/Absent')
os.system('mkdir ' + topDirName + '/Present')

for sample in allSamples: 
    if 'Negative' in sample or 'Sat' in sample:
        cmdString = "mv " + sample + " " + topDirName + "/Absent"
    else:
        cmdString = "mv " + sample + " " + topDirName + "/Present"
    os.system(cmdString)

Each directory within the classification subdirectory contains a series of folders that have the following contents:

1) A .txt file containing some timing and other metadata about the app run.

2) A folder called 'Photos' which contains the sequence of images acquired.

The next cell will obliterate the middle directory (the directory just below the classification directory) and repackage each run's photos into a uniquely identifiable folder of its own (lacking the text file) at the level where the middle directories once sat.



In [13]:
### Find 'vestigial' directories to be deleted after unpacking.
presentVestDirs = np.array(glob(topDirName + '/Present/*'))
absentVestDirs = np.array(glob(topDirName + '/Absent/*'))

# Unpack the JPG photo files into identifiable directories.
allPresentSamples = np.array(glob(topDirName + '/Present/*/*'))
for sample in allPresentSamples:
    # Grab the unique identifiers -- phone model, sample # and date/time
    digestedSample = re.split('/',sample)
    digestedTimestamp = re.split('_',digestedSample[3])
    newDir = digestedSample[2] + '_' + digestedTimestamp[1] + '_' + digestedTimestamp[2]
    # Make the new directory
    cmdString = 'mkdir ' + topDirName + '/Present/' + newDir
    os.system(cmdString)
    # Add the photos directly to it
    cmdString = 'mv ' + sample + '/Photos/*' + ' ' + topDirName + '/Present/' + newDir
    os.system(cmdString)
    
allAbsentSamples = np.array(glob(topDirName + '/Absent/*/*'))
for sample in allAbsentSamples:
    # Grab the unique identifiers -- phone model, sample # and date/time
    digestedSample = re.split('/',sample)
    digestedTimestamp = re.split('_',digestedSample[3])
    newDir = digestedSample[2] + '_' + digestedTimestamp[1] + '_' + digestedTimestamp[2]
    # Make the new directory
    cmdString = 'mkdir ' + topDirName + '/Absent/' + newDir
    os.system(cmdString)
    # Add the photos directly to it
    cmdString = 'mv ' + sample + '/Photos/*' + ' ' + topDirName + '/Absent/' + newDir
    os.system(cmdString)
    
    
# Delete vestigial directories.
for direc in presentVestDirs:
    os.system('rm -rf ' + direc)
    
for direc in absentVestDirs:
    os.system('rm -rf ' + direc)

In [15]:
# Get the filenames.
imgSeqsToCorrect = os.popen('cd ~/src/YPB-AI/; find -name "*IMG_9999.JPG"').read()
# Split into unique entities.
imgSeqsToCorrect = re.split('\n',imgSeqsToCorrect)
# Drop the last entry, it's empty.
imgSeqsToCorrect.pop();

# Change any files that don't start with 'IMG_9' (i.e., any files that come after the wraparound) to start with 'J'.
# This will mean that when they are sorted, they will come correctly after the wraparound.
for sample in imgSeqsToCorrect:
    digestedSample = re.split('/',sample)
    pathToDir = os.path.join(digestedSample[0],digestedSample[1],digestedSample[2],digestedSample[3])
    filesInDir = os.listdir(pathToDir)
    for i in range(len(filesInDir)):
        if 'IMG_9' not in filesInDir[i]:
            newFile = filesInDir[i].replace('I','J',1)
            cmdString = 'mv ' + os.path.join(pathToDir,filesInDir[i]) + ' ' + os.path.join(pathToDir,newFile)
            os.system(cmdString)

# Change the images in the folder into a stack of 12-image input

In [3]:
import torch
import torchvision.models as models
from PIL import Image
import pickle

In [17]:
topdir = topDirName
ndx = 0
topdirs = [direc for direc in os.listdir(topdir) if direc[0] is not '.']

# Calculate the total number of stacks to transform.
numStacks = 0
for classDir in topdirs:
    thisClassDir = os.path.join(topdir,classDir)
    directories = [direc for direc in os.listdir(thisClassDir) if direc[0] is not '.']
    numStacks += len(directories)

for classDir in topdirs:
    thisClassDir = os.path.join(topdir,classDir)
    directories = [direc for direc in os.listdir(thisClassDir) if direc[0] is not '.']


    # Do the transformation.
    for photoDir in directories:

        thisPhotoDir = os.path.join(thisClassDir, photoDir)
        ##pin pin fix##

#             print(thisPhotoDir)
#             print(os.path.isfile(photoDir))
#             continue

        if not os.path.isdir(thisPhotoDir):
            continue

        ## end ##
        files = [photofile for photofile in os.listdir(thisPhotoDir) if photofile.lower().endswith('jpg')]
        print("Transforming image stack {}".format(thisPhotoDir))
        files.sort()
        output = []
        # Make sure to appropriately transform the image size and normalize the RGB values, as the model expects. 
        for file in files:
            thisFile = os.path.join(thisPhotoDir,file)
            imgArr = np.float32(np.array(Image.open(thisFile).resize((32,32))).reshape([1,3,32,32]))
            imgArr[0][0] = ((imgArr[0][0] - 123.675)/58.395)
            imgArr[0][1] = ((imgArr[0][1] - 116.28)/57.12)
            imgArr[0][2] = ((imgArr[0][2] - 103.53)/57.375)
            output.append(imgArr)

        FinalOutput = torch.FloatTensor(output)
        # for i in range(len(files)):
        #    FinalOutput[i] = output[i]
        pathStr = thisPhotoDir + '.pickle'
        with open(pathStr,'wb') as f:
            pickle.dump(FinalOutput,f)
            ndx += 1
            print("{}% complete.".format(ndx/numStacks*100))
    

Transforming image stack bamData/Absent/6S_Negative_Sample3_Light_16-Apr-2019_15-59-47
0.007203054094936254% complete.
Transforming image stack bamData/Absent/XS_Sat2_07-Jun-2019_20-13-37
0.014406108189872507% complete.
Transforming image stack bamData/Absent/6_Negative_Sample3_Back_Light_16-Apr-2019_13-17-37
0.02160916228480876% complete.
Transforming image stack bamData/Absent/7_redBox_Spaced_Negative_06-Nov-2019_12-01-55
0.028812216379745015% complete.
Transforming image stack bamData/Absent/SE_Negative_Dark_15-Mar-2019_15-37-50
0.03601527047468126% complete.
Transforming image stack bamData/Absent/XS_Sat_17-Apr-2019_11-36-09
0.04321832456961752% complete.
Transforming image stack bamData/Absent/6_Negative_Sample3_Back_Light_16-Apr-2019_14-11-36
0.05042137866455376% complete.
Transforming image stack bamData/Absent/6_Negative_Dark_13-Mar-2019_13-01-20
0.05762443275949003% complete.
Transforming image stack bamData/Absent/XS_Sat2_07-Jun-2019_16-46-00
0.06482748685442628% complete.
Tr

0.569041273499964% complete.
Transforming image stack bamData/Absent/7_Negative_Dark_13-Mar-2019_17-49-17
0.5762443275949002% complete.
Transforming image stack bamData/Absent/7_Negative_Dark_13-Mar-2019_15-43-48
0.5834473816898365% complete.
Transforming image stack bamData/Absent/6_Negative_Sample3_Back_Light_16-Apr-2019_14-35-57
0.5906504357847727% complete.
Transforming image stack bamData/Absent/XS_Sat_07-Jun-2019_14-27-08
0.597853489879709% complete.
Transforming image stack bamData/Absent/7_Negative_Sample3_Light_16-Apr-2019_17-51-45
0.6050565439746453% complete.
Transforming image stack bamData/Absent/6_Negative_Dark_13-Mar-2019_14-01-40
0.6122595980695815% complete.
Transforming image stack bamData/Absent/6S_Negative_Sample3_Light_16-Apr-2019_16-33-35
0.6194626521645178% complete.
Transforming image stack bamData/Absent/7_Negative_Dark_13-Mar-2019_16-36-47
0.626665706259454% complete.
Transforming image stack bamData/Absent/6S_Negative_Dark_14-Mar-2019_11-34-07
0.6338687603543

1.1164733847151194% complete.
Transforming image stack bamData/Absent/6S_Sat_03-Jun-2019_15-11-25
1.1236764388100555% complete.
Transforming image stack bamData/Absent/6_Negative_Sample3_Back_Light_16-Apr-2019_13-06-09
1.1308794929049917% complete.
Transforming image stack bamData/Absent/XS_Sat2_07-Jun-2019_18-34-32
1.138082546999928% complete.
Transforming image stack bamData/Absent/6S_Negative_Sample3_Light_16-Apr-2019_15-34-08
1.1452856010948642% complete.
Transforming image stack bamData/Absent/6S_filledTube_Negative_05-Nov-2019_14-30-07
1.1524886551898004% complete.
Transforming image stack bamData/Absent/6S_Negative_Random_03-Jun-2019_14-20-53
1.1596917092847367% complete.
Transforming image stack bamData/Absent/XS_Negative_Random_30-May-2019_15-32-34
1.166894763379673% complete.
Transforming image stack bamData/Absent/7_Negative_Sample3_Light_16-Apr-2019_18-04-09
1.174097817474609% complete.
Transforming image stack bamData/Absent/XS_Negative_Dark_14-Mar-2019_18-59-17
1.18130087

1.6711085500252107% complete.
Transforming image stack bamData/Absent/7_Negative_Dark_13-Mar-2019_16-36-16
1.678311604120147% complete.
Transforming image stack bamData/Absent/7_greenLabel_Spaced_Negative_06-Nov-2019_14-58-30
1.6855146582150833% complete.
Transforming image stack bamData/Absent/7_redBox_Spaced_Negative_06-Nov-2019_11-13-52
1.6927177123100194% complete.
Transforming image stack bamData/Absent/XS_Sat_07-Jun-2019_14-24-52
1.6999207664049558% complete.
Transforming image stack bamData/Absent/XS_Sat2_07-Jun-2019_19-53-10
1.707123820499892% complete.
Transforming image stack bamData/Absent/SE_Negative_Random_03-Jun-2019_16-37-51
1.7143268745948281% complete.
Transforming image stack bamData/Absent/7_Negative_Random_30-May-2019_17-08-25
1.7215299286897645% complete.
Transforming image stack bamData/Absent/6S_Negative_Dark_14-Mar-2019_10-59-50
1.7287329827847007% complete.
Transforming image stack bamData/Absent/6_Negative_Dark_13-Mar-2019_12-48-54
1.7359360368796368% complete

2.2329467694302387% complete.
Transforming image stack bamData/Absent/XS_Sat2_07-Jun-2019_18-49-14
2.240149823525175% complete.
Transforming image stack bamData/Absent/XS_Negative_Dark_14-Mar-2019_18-27-25
2.247352877620111% complete.
Transforming image stack bamData/Absent/7_redBox_Angle_Negative_06-Nov-2019_12-15-01
2.254555931715047% complete.
Transforming image stack bamData/Absent/6_Negative_Dark_13-Mar-2019_11-30-24
2.2617589858099834% complete.
Transforming image stack bamData/Absent/XS_Negative_Random_04-Jun-2019_10-10-02
2.2689620399049195% complete.
Transforming image stack bamData/Absent/6S_Negative_Dark_14-Mar-2019_10-35-51
2.276165093999856% complete.
Transforming image stack bamData/Absent/SE_Negative_Random_03-Jun-2019_16-38-20
2.2833681480947923% complete.
Transforming image stack bamData/Absent/6S_Negative_Sample3_Light_16-Apr-2019_15-33-33
2.2905712021897284% complete.
Transforming image stack bamData/Absent/XS_Sat_07-Jun-2019_14-48-28
2.2977742562846646% complete.
Tr

...

97.52214939134193% complete.
Transforming image stack bamData/Present/7_Sample1_Dark_12-Mar-2019_14-23-23
97.52935244543687% complete.
Transforming image stack bamData/Present/7_fiiledTube_05-Nov-2019_13-05-45
97.5365554995318% complete.
Transforming image stack bamData/Present/6S_Distance_and_Curved_13-Jun-2019_16-44-16
97.54375855362673% complete.
Transforming image stack bamData/Present/XS_Sample3_Dark_12-Mar-2019_16-29-44
97.55096160772167% complete.
Transforming image stack bamData/Present/XS_Sample5_Light_27-Mar-2019_17-13-46
97.55816466181662% complete.
Transforming image stack bamData/Present/SE_Sample3_Light_28-Mar-2019_15-46-17
97.56536771591155% complete.
Transforming image stack bamData/Present/6S_Sample5_Dark_08-Mar-2019_13-36-15
97.57257077000648% complete.
Transforming image stack bamData/Present/SE_Sample5_Light_01-Apr-2019_16-00-46
97.57977382410142% complete.
Transforming image stack bamData/Present/6S_redBottle_H_29-Oct-2019_10-52-51
97.58697687819635% complete.
Tran

98.07678455665202% complete.
Transforming image stack bamData/Present/6S_Sample5_Light_27-Mar-2019_10-50-29
98.08398761074696% complete.
Transforming image stack bamData/Present/XS_filledTube_05-Nov-2019_10-09-24
98.09119066484189% complete.
Transforming image stack bamData/Present/6S_Sample5_Light_27-Mar-2019_11-03-30
98.09839371893683% complete.
Transforming image stack bamData/Present/XS_Distance_and_Curved_13-Jun-2019_11-49-02
98.10559677303176% complete.
Transforming image stack bamData/Present/6_Sample1_Dark_12-Mar-2019_18-26-25
98.11279982712671% complete.
Transforming image stack bamData/Present/6_Sample3_Light_25-Mar-2019_14-57-53
98.12000288122164% complete.
Transforming image stack bamData/Present/SE_Sample3_Dark_11-Mar-2019_12-03-22
98.12720593531658% complete.
Transforming image stack bamData/Present/6_Sample2_Dark_08-Mar-2019_17-20-54
98.13440898941151% complete.
Transforming image stack bamData/Present/XS_Sample3_Light_28-Mar-2019_14-03-06
98.14161204350644% complete.
Tr

98.63862277605705% complete.
Transforming image stack bamData/Present/XS_Sample2_Dark_12-Mar-2019_16-11-20
98.64582583015199% complete.
Transforming image stack bamData/Present/7_Sample5_Dark_12-Mar-2019_11-53-47
98.65302888424692% complete.
Transforming image stack bamData/Present/SE_Sample4_Dark_11-Mar-2019_12-59-04
98.66023193834185% complete.
Transforming image stack bamData/Present/6S_Sample5_Light_27-Mar-2019_11-05-27
98.6674349924368% complete.
Transforming image stack bamData/Present/7_Sample2_Dark_12-Mar-2019_13-36-15
98.67463804653173% complete.
Transforming image stack bamData/Present/6_Sample5_Light_26-Mar-2019_18-08-47
98.68184110062667% complete.
Transforming image stack bamData/Present/XS_Sample1_Dark_12-Mar-2019_15-39-24
98.6890441547216% complete.
Transforming image stack bamData/Present/6_Sample5_Light_26-Mar-2019_18-07-43
98.69624720881653% complete.
Transforming image stack bamData/Present/6S_Sample2_Dark_08-Mar-2019_11-01-59
98.70345026291147% complete.
Transformin

99.20766404955701% complete.
Transforming image stack bamData/Present/7_fiiledTube_05-Nov-2019_12-54-14
99.21486710365195% complete.
Transforming image stack bamData/Present/SE_Sample5_Dark_11-Mar-2019_14-40-25
99.22207015774688% complete.
Transforming image stack bamData/Present/6S_Sample4_Dark_08-Mar-2019_13-10-25
99.22927321184181% complete.
Transforming image stack bamData/Present/7_Sample3_Light_27-Mar-2019_14-10-13
99.23647626593676% complete.
Transforming image stack bamData/Present/7_Sample3_Light_27-Mar-2019_15-06-29
99.2436793200317% complete.
Transforming image stack bamData/Present/6_Sample3_Dark_08-Mar-2019_16-09-10
99.25088237412663% complete.
Transforming image stack bamData/Present/6_Sample5_Light_26-Mar-2019_16-33-45
99.25808542822156% complete.
Transforming image stack bamData/Present/7_fiiledTube_05-Nov-2019_12-33-16
99.2652884823165% complete.
Transforming image stack bamData/Present/6_Sample5_Dark_08-Mar-2019_14-12-46
99.27249153641144% complete.
Transforming image

99.77670532305697% complete.
Transforming image stack bamData/Present/7_Sample4_Dark_12-Mar-2019_12-30-12
99.7839083771519% complete.
Transforming image stack bamData/Present/SE_Sample3_Light_28-Mar-2019_15-59-53
99.79111143124685% complete.
Transforming image stack bamData/Present/SE_Sample1_Dark_07-Mar-2019_16-11-31
99.79831448534179% complete.
Transforming image stack bamData/Present/6S_pillBottle_H_29-Oct-2019_12-26-06
99.80551753943672% complete.
Transforming image stack bamData/Present/SE_Sample5_Light_01-Apr-2019_16-17-19
99.81272059353165% complete.
Transforming image stack bamData/Present/7_Sample5_Dark_12-Mar-2019_12-02-06
99.8199236476266% complete.
Transforming image stack bamData/Present/6S_filledBottle_H_29-Oct-2019_15-58-54
99.82712670172153% complete.
Transforming image stack bamData/Present/7_Distance_and_Curved_13-Jun-2019_15-08-13
99.83432975581647% complete.
Transforming image stack bamData/Present/6_Sample3_Light_25-Mar-2019_14-22-33
99.8415328099114% complete.
Tra

Define the function that will transform model output into a pickle file.

In [18]:
def transform_to_pickle(model,topdir):
    model.eval()
    ndx = 0
    topdirs = [direc for direc in os.listdir(topdir) if direc[0] is not '.']
    
    # Calculate the total number of stacks to transform.
    numStacks = 0
    for classDir in topdirs:
        thisClassDir = os.path.join(topdir,classDir)
        directories = [direc for direc in os.listdir(thisClassDir) if direc[0] is not '.']
        numStacks += len(directories)
    
    for classDir in topdirs:
        thisClassDir = os.path.join(topdir,classDir)
        directories = [direc for direc in os.listdir(thisClassDir) if direc[0] is not '.']
        
        
        # Do the transformation.
        for photoDir in directories:
            
            thisPhotoDir = os.path.join(thisClassDir, photoDir)
            ##pin pin fix##
            
#             print(thisPhotoDir)
#             print(os.path.isfile(photoDir))
#             continue
            
            if not os.path.isdir(thisPhotoDir):
                continue

            ## end ##
            files = [photofile for photofile in os.listdir(thisPhotoDir) if photofile.lower().endswith('jpg')]
            print("Transforming image stack {}".format(thisPhotoDir))
            files.sort()
            output = []
            # Make sure to appropriately transform the image size and normalize the RGB values, as the model expects. 
            for file in files:
                thisFile = os.path.join(thisPhotoDir,file)
                imgArr = np.float32(np.array(Image.open(thisFile).resize((224,224))).reshape([1,3,224,224]))
                imgArr[0][0] = ((imgArr[0][0] - 123.675)/58.395)
                imgArr[0][1] = ((imgArr[0][1] - 116.28)/57.12)
                imgArr[0][2] = ((imgArr[0][2] - 103.53)/57.375)
                if torch.cuda.is_available():
                    output.append(model(torch.cuda.FloatTensor(imgArr)))
                else:
                    output.append(model(torch.FloatTensor(imgArr)))

            FinalOutput = imgArr
            pathStr = thisPhotoDir + '.pickle'
            with open(pathStr,'wb') as f:
                pickle.dump(FinalOutput,f)
                ndx += 1
                print("{}% complete.".format(ndx/numStacks*100))
    

Do the transformation

In [22]:
transform_to_pickle(model,topDirName)

NameError: name 'model' is not defined

Clean up the directory, removing everything but the pickle files.

In [23]:
# Assemble all the pickle files and remove the rest.
classDirs = [direc for direc in os.listdir(topDirName) if direc[0] is not '.']
for classDir in classDirs:
    newDir = os.path.join(topDirName,classDir)
    fileDirs = [direc for direc in os.listdir(newDir) if direc[0] is not '.' and not direc.lower().endswith('.pickle')]
    for fileDir in fileDirs:
        pickleFileDir = os.path.join(newDir,fileDir)
        os.system('rm -rf ' + pickleFileDir)

# Split the directory into train/test/validation sets

In [4]:
import random

In [7]:
def split_folders(inputDir, outputDir, trainValTestRatio, seed):
    # Assume inputDir is organized as 'inputDir/class/topFolder/photos'
    classes = os.listdir(inputDir)
    # Exclude folders where '.' is the first character.
    classes = [directory for directory in classes if directory[0] != '.']
    numClasses = len(classes)
    # Create output directories
    os.system('mkdir ' + outputDir)
    os.system('mkdir ' + outputDir + '/train')
    os.system('mkdir ' + outputDir + '/valid')
    os.system('mkdir ' + outputDir + '/test')
    # Pull files and put into the appropriate folders.
    for directory in classes:
        files = os.listdir(os.path.join(inputDir, directory))
        files.sort()
        random.seed(seed)
        random.shuffle(files)
        numFiles = len(files)
        # Get the indices.
        ndxAmts = np.intc((np.round(trainValTestRatio / np.sum(trainValTestRatio) * numFiles)))
        numNdxes = np.sum(ndxAmts)
        if numNdxes != numFiles:
            ndxAmts[0] += numFiles-numNdxes
        trainFiles = files[0:ndxAmts[0]]
        valFiles = files[ndxAmts[0]:ndxAmts[0]+ndxAmts[1]]
        testFiles = files[ndxAmts[0]+ndxAmts[1]:]
        # Put the training directories in the appropriate folder
        # First make the class directories.
        os.system('mkdir ' + outputDir + '/train/' + directory)
        os.system('mkdir ' + outputDir + '/valid/' + directory)
        os.system('mkdir ' + outputDir + '/test/' + directory)
        for file in trainFiles:
            os.system('cp -r ' + inputDir + '/' + directory + '/' + file + ' ' + outputDir + '/train/' + directory + '/' + file)
        for file in valFiles:
            os.system('cp -r ' + inputDir + '/' + directory + '/' + file + ' ' + outputDir + '/valid/' + directory + '/' + file)
        for file in testFiles:
            os.system('cp -r ' + inputDir + '/' + directory + '/' + file + ' ' + outputDir + '/test/' + directory + '/' + file)

In [5]:
finalDirName = 'Split_Data_bamData_no_pillBottle'

In [6]:
for idx in range(5):
    split_folders(topDirName, finalDirName_root + str(idx),[0.8,0.1,0.1],1000 + idx)

NameError: name 'split_folders' is not defined

In [8]:
split_folders(topDirName, finalDirName,[0.8,0.1,0.1],7778)